In [33]:
import zipfile
import numpy as np

import glob
import re
import pandas as pd

# read the dataset using the compression zip
all_data = pd.read_csv('all_data_for_analysis.zip',compression='zip')

In [34]:
pd.set_option('display.max_rows', 100)

In [35]:
all_data['Count'] = 1

Drop nulls in comments

In [36]:
all_data['Comment'].isna().sum()

2

In [37]:
all_data = all_data.dropna(subset=['Comment'])

In [38]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid,Count
0,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27,1
1,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62,1
2,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74,1
3,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33,1
4,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30096,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28,83.32,1
30097,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113,94.15,1
30098,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95,72.32,1
30099,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77,1


In [39]:
only_merged = all_data[all_data['status']==1]

In [40]:
only_abandoned = all_data[all_data['status']==0]

In [41]:
only_abandoned.count()

PR_number                    6427
Comment                      6427
Author                       6427
Date                         6427
Commit_message               4195
Descriptions                 6425
status                       6427
main_author                  6427
revision_number              6427
url                          6427
PR_number_no_patch_number    6427
comment_length               6427
Flesch-Kincaid               6427
Count                        6427
dtype: int64

In [42]:
only_merged.count()

PR_number                    23672
Comment                      23672
Author                       23672
Date                         23672
Commit_message               16977
Descriptions                 23672
status                       23672
main_author                  23672
revision_number              23672
url                          23672
PR_number_no_patch_number    23672
comment_length               23672
Flesch-Kincaid               23672
Count                        23672
dtype: int64

Join all comments from PRs that have been merged

In [43]:
all_comments = ''.join(only_abandoned['Comment'])

In [44]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GNR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
r = Rake()
r.extract_keywords_from_text(all_comments)

In [46]:
dict(sorted(r.get_word_degrees().items(), key=lambda item: item[1], reverse=True))

{'abandoned': 3748,
 'commit': 2266,
 'good': 1827,
 'looks': 1826,
 'branch': 1745,
 'change': 1652,
 '1': 1647,
 'would': 1605,
 'rebase': 1465,
 'script': 1324,
 'patchset': 1323,
 'patch': 1306,
 '->': 1305,
 'trivial': 1304,
 'detection': 1302,
 'comment': 1297,
 'inline': 1140,
 'submit': 1120,
 'someone': 1020,
 'villegas': 1019,
 'roselyn': 1014,
 'prefer': 1013,
 'else': 1009,
 'must': 1004,
 'new': 1003,
 'message': 928,
 'merge': 921,
 'abandonedwrong': 901,
 'id': 852,
 'previous': 810,
 'test': 796,
 'one': 776,
 'like': 735,
 'please': 674,
 'matches': 672,
 'abandonedduplicate': 654,
 'well': 621,
 'also': 612,
 'eclipse': 599,
 'review': 594,
 'need': 587,
 'java': 554,
 'code': 543,
 'restricted': 533,
 'think': 526,
 'use': 500,
 'line': 496,
 'set': 488,
 '...': 484,
 'approve': 478,
 'changes': 474,
 'still': 473,
 '2': 459,
 'gerrit': 450,
 'org': 448,
 'approved': 441,
 '0': 426,
 'jgit': 425,
 'see': 416,
 ').': 416,
 'tests': 403,
 'fix': 401,
 'get': 393,
 'add

# Sentiment Analysis

In [47]:
from textblob import TextBlob

In [48]:
all_data['Polarity'] = 0
all_data['Subjectivity'] = 0
for n in range(len(all_data)):
    all_data['Polarity'].iloc[n] = TextBlob(all_data['Comment'].iloc[n]).sentiment.polarity
    all_data['Subjectivity'].iloc[n] = TextBlob(all_data['Comment'].iloc[n]).sentiment.subjectivity

C:\Users\GNR\AppData\Local\Temp\ipykernel_9268\1903632213.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Polarity'].iloc[n] = TextBlob(all_data['Comment'].iloc[n]).sentiment.polarity
C:\Users\GNR\AppData\Local\Temp\ipykernel_9268\1903632213.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Subjectivity'].iloc[n] = TextBlob(all_data['Comment'].iloc[n]).sentiment.subjectivity


In [49]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid,Count,Polarity,Subjectivity
0,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27,1,0.000000,0.362500
1,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62,1,0.000000,1.000000
2,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74,1,0.000000,0.000000
3,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33,1,0.068750,0.418750
4,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17,1,0.118182,0.677273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30096,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28,83.32,1,0.000000,0.000000
30097,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113,94.15,1,0.000000,0.000000
30098,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95,72.32,1,0.000000,0.000000
30099,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77,1,0.000000,0.000000


## Means

In [50]:
metrics_per_pull_request = all_data.groupby(['PR_number_no_patch_number']).sum()
metrics_per_pull_request_merged = only_merged.groupby(['PR_number_no_patch_number']).sum()
metrics_per_pull_request_abandoned = only_abandoned.groupby(['PR_number_no_patch_number']).sum()
metrics_per_pull_request

,status,revision_number,comment_length,Flesch-Kincaid,Count,Polarity,Subjectivity
PR_number_no_patch_number,,,,,,,
10003,6,24,544,518.82,6,1.400000,1.200000
10004,7,39,1268,492.87,7,-0.088505,3.109008
10006,1,1,27,100.24,1,0.700000,0.600000
10011,1,1,27,100.24,1,0.700000,0.600000
10012,1,1,8,36.62,1,0.000000,0.000000
...,...,...,...,...,...,...,...
9987_,1,1,8,36.62,1,0.000000,0.000000
99914,0,5,521,421.48,5,0.987302,1.767857
9991_,3,3,312,219.85,3,0.925000,1.400000


In [51]:
metrics_per_pull_request.mean()

status               1.745723
revision_number      7.350664
comment_length     229.100959
Flesch-Kincaid     140.961628
Count                2.219690
Polarity             0.552052
Subjectivity         0.828747
dtype: float64

In [52]:
metrics_per_pull_request_merged.mean()

status               2.211097
revision_number      7.454325
comment_length     226.210256
Flesch-Kincaid     144.966543
Count                2.211097
dtype: float64

In [53]:
metrics_per_pull_request_abandoned.mean()

status               0.000000
revision_number      6.910957
comment_length     238.192000
Flesch-Kincaid     125.018390
Count                2.235478
dtype: float64